# FC25 Rush Detection Model
## Deploy as a Service
This notebook will deploy the Rush detection model as a service. The goal is to leverage as much project code as possible, and to keep this notebook as lightweight as possible.

Ideally, this notebook will be run on a local subnet, and the service will be accessed via a local IP address. Alternatively, it could be run on colab, and accessed via a public address.

### Setup
From your colab notebook, you should be able to run the following commands to setup the environment.

```
%%capture
%%sh
git clone https://github.com/swaynos/urban-palm-tree.git
pip install poetry

%cd urban-palm-tree

%%sh
poetry install
```

In [ ]:
import sys
import os

# Check if the current working directory is under 'notebooks'
if 'notebooks' in os.getcwd().split(os.sep):
    # Adjust the path to include the 'src' directory
    sys.path.append(os.path.abspath(os.path.join('..', 'src')))  # This adds the correct src path

# Verify the current system path
print(sys.path)

## Run the Flask App from Local Machine

In [ ]:
import asyncio
from flask import Flask, request, jsonify
from inference.yolo_object_detector import YoloObjectDetector
from inference.rush_inference import parse_rush_model_results
from utilities import config
from PIL import Image

# Create a Flask app
app = Flask(__name__)

PORT = 8080

# Initialize the detector
detector = YoloObjectDetector(config.HF_RUSH_DETECTION_PATH, config.HF_RUSH_DETECTION_FILENAME)

# Define a route for the inference
@app.route('/detect', methods=['POST'])
def detect_objects():
    # Check if an image file was provided
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        # Open the image file
        image = Image.open(file)

        # Run the detector. Keep in mind this is a syncrhonous operation
        results = asyncio.run(detector.detect_objects(image, parse_rush_model_results))
        
        # Return the results as JSON
        return jsonify(results), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=PORT)

## Run the Flask App from Google Colab

In [ ]:
# If running on Colab
!pip install pyngrok
from pyngrok import ngrok
import os

# Authenticate ngrok with your token
NGROK = ""  # Replace with your ngrok token
ngrok.set_auth_token(NGROK)

# Define a function to start Flask
def run_app():
    os.system(f"python3 -m flask run --host=0.0.0.0 --port={PORT}")

# Start Flask in the background
import threading
threading.Thread(target=run_app).start()

# Step 5: Set up ngrok tunnel
public_url = ngrok.connect(PORT)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{PORT}\"")